# Model

In [1]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical


<h3 style= "color:blue;"> Importing Data Set </h3>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# load your dataframe
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Research/Method 2 - Titles/preprocessed_title.csv",index_col=[0])

In [4]:
# change both types into string
df['Title'] = df['Title'].astype('str')

In [5]:
texts = df['Title'].values
labels = df['classes'].values

# Split the data into training and testing sets
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.3, random_state=42)

In [6]:
# Initialize the tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_texts)

# Convert text to sequences
train_sequences = tokenizer.texts_to_sequences(train_texts)
test_sequences = tokenizer.texts_to_sequences(test_texts)

# Pad sequences
max_length = 300  # maximum sequence length
train_data = pad_sequences(train_sequences, maxlen=max_length)
test_data = pad_sequences(test_sequences, maxlen=max_length)

In [7]:
num_classes = max(df['classes']) + 1  # Calculate the number of classes
# Determine vocabulary size
vocab_size = len(tokenizer.word_index) + 1
# Define the embedding dimension
embedding_dim = 100  # Adjust the value as needed


# Convert the label values to categorical format
train_labels = to_categorical(train_labels, num_classes=num_classes)
test_labels = to_categorical(test_labels, num_classes=num_classes)

# Define the LSTM model
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))
model.add(LSTM(units=64))
model.add(Dense(units=num_classes, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print the model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 300, 100)          143600    
                                                                 
 lstm (LSTM)                 (None, 64)                42240     
                                                                 
 dense (Dense)               (None, 12)                780       
                                                                 
Total params: 186,620
Trainable params: 186,620
Non-trainable params: 0
_________________________________________________________________


In [8]:
# Train the model
model.fit(train_data, train_labels, epochs=10, batch_size=64, validation_data=(test_data, test_labels))

Epoch 1/10
9/9 [==============================] - 9s 698ms/step - loss: 2.4660 - accuracy: 0.1741 - val_loss: 2.4079 - val_accuracy: 0.2562
Epoch 2/10
9/9 [==============================] - 4s 397ms/step - loss: 2.3721 - accuracy: 0.2114 - val_loss: 2.2749 - val_accuracy: 0.2562
Epoch 3/10
9/9 [==============================] - 3s 385ms/step - loss: 2.2927 - accuracy: 0.2114 - val_loss: 2.2596 - val_accuracy: 0.2562
Epoch 4/10
9/9 [==============================] - 3s 390ms/step - loss: 2.2614 - accuracy: 0.2114 - val_loss: 2.2465 - val_accuracy: 0.2562
Epoch 5/10
9/9 [==============================] - 6s 630ms/step - loss: 2.2161 - accuracy: 0.2185 - val_loss: 2.2203 - val_accuracy: 0.2562
Epoch 6/10
9/9 [==============================] - 4s 395ms/step - loss: 2.1482 - accuracy: 0.2256 - val_loss: 2.1869 - val_accuracy: 0.2562
Epoch 7/10
9/9 [==============================] - 3s 394ms/step - loss: 2.0520 - accuracy: 0.2966 - val_loss: 2.1656 - val_accuracy: 0.3058
Epoch 8/10
9/9 [====

In [9]:
# Evaluate the model
loss, accuracy = model.evaluate(test_data, test_labels)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

8/8 [==============================] - 0s 46ms/step - loss: 2.0786 - accuracy: 0.2975
Test Loss: 2.0786149501800537
Test Accuracy: 0.2975206673145294


In [10]:
from sklearn.metrics import classification_report

# Assuming you have already trained your model and obtained predictions
predictions = model.predict(test_data)
predicted_labels = np.argmax(predictions, axis=1)

# Generate classification report
report = classification_report(np.argmax(test_labels, axis=1), predicted_labels)

print(report)

8/8 [==============================] - 1s 71ms/step
              precision    recall  f1-score   support

           0       0.67      0.20      0.31        10
           1       0.00      0.00      0.00        12
           2       0.00      0.00      0.00         7
           3       0.06      0.04      0.05        27
           4       0.51      0.68      0.58        62
           5       0.21      0.45      0.29        29
           6       0.20      0.04      0.07        24
           7       0.00      0.00      0.00        13
           8       0.00      0.00      0.00         8
           9       0.00      0.00      0.00        16
          10       0.19      0.52      0.27        25
          11       0.00      0.00      0.00         9

    accuracy                           0.30       242
   macro avg       0.15      0.16      0.13       242
weighted avg       0.23      0.30      0.24       242



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
